# Projet:Customer Churn (Article 2)

# Partie 1 : Exploration de la base de données

## Analyse de forme

### 1-Importation des bibliothèques

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### 2-Importation des données

In [5]:
data=pd.read_excel('Telco_customer_churn.csv.xlsx')
data

FileNotFoundError: [Errno 2] No such file or directory: 'Telco_customer_churn.csv.xlsx'

In [ ]:
df=data.copy()

### 3-Dimension de la base de donnée Telco_Customer_Churn

In [ ]:
print("Les dimensions de la bade de données sont : " )
print(" * Nonmbre de lignes est :", df.shape[0] )
print(" * Nonmbre de colonnes est :", df.shape[1] )

### 4-Types de variables dans la base de données

In [ ]:
print(df.dtypes)
df.dtypes.value_counts()
#Notre base de données comporte des variables quantitative et qualitative

### 5-Valeurs nuls

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(df.isna(), cbar=False)
#affficher les données manquantes (ou il y a du blanc on a des données manquantes)

In [ ]:
(df.isna().sum()/df.shape[0]).sort_values(ascending=True)#pourcentages des valeurs nulles ordonées
# On verifie l'existance des valeurs nulles
#En effet la variable Churn reseason contient 5174 Valeur nuls

##### Probleme de la colonne Totals Charges

In [ ]:
t=len(df[df['Total Charges'] == " "])#on voit pas ce probleme avec sum() car Total charges est type object 
print(t)

In [ ]:
df.drop(df[df['Total Charges'] == " "].index, inplace=True)#suppression des cases vides 
df['Total Charges'] = df['Total Charges'].astype('float64')#conversion de Total charges 
#en float car il etait initallement de type object

### 6-unicité des features

In [ ]:
for col in df.columns:
    if df[col].dtype=='O':
        print(col)
        print(df[col].unique())
        
### Verifier l'unicité des contenus des variables de types Objet 

Remarque :
La base de données contient des colonnes non informatives par rapport au phenomene de desabonnement telques:
CustomerID, Count(toutes les valeurs sont à 1), Country(Tous USA), State(Tous California),zip Code ,Lat Long ,Latitude, Longitude , Churn Label (Equivalante à Churn Value),Churn Reason

## Analyse de fond

### 1. Visulation initiale - Elimination des colonnes inutiles

In [ ]:
df = df.drop(['CustomerID', 'Count', 'Country','State', 
              'Zip Code', 'Lat Long','Churn Label', 'Churn Reason', 
              'Lat Long', 'Latitude', 'Longitude','City','CLTV','Churn Score'], axis=1)
#Suppression des colonnes non informatives

In [ ]:
#for col in df.select_dtypes('object'):
  #  a=len(df[df[col] == " "])
   # print(a)
#on a plus de valeurs nulles
#for col in df.select_dtypes('int64'):
  #  a=len(df[df[col] == " "])
   # print(a)

In [ ]:
missing = df.isnull().sum()
missing[missing > 0].count()
#on a plus de colonnes avec des valeurs manquantes

Les colonnes qui restent sont:

In [ ]:
df.dtypes

### 2- Examen de la colonne target

on va appeler les clinets qui ont quitter le dernier mois Churning Customers et autres Non-Churning Customers

In [ ]:
df['Churn Value'].value_counts(normalize=True)*100#nomalizepour afficher les resultats en pourcentage

In [ ]:
sns.countplot(data=df, x="Churn Value")
plt.title('Churn Distribution')
plt.show()

seulement 26.5% des clients ont quitter le dernier mois donc une une base de données non equilibrée.Donc on devrait 
choisir une  une bonne métrique ,Accuracy dans ce cas n'est plus adaptée.

### 3-Histogrames des valeurs quantitatives

In [ ]:
for col in df.select_dtypes('int64'):
    plt.figure()
    sns.distplot(df[col])

In [ ]:
for col in df.select_dtypes('float'):
    plt.figure()
    sns.distplot(df[col])

### 4-variables qualitatives

In [ ]:
for col in df.select_dtypes('object'):
    print(f'{col :-<50} {df[col].unique()}')

In [ ]:
for col in df.select_dtypes('object'):
    plt.figure()
    df[col].value_counts().plot.pie()

## 5- Relation Target/Variables

In [ ]:
df.info()

### a- Variables quantitatives

In [ ]:
num_features = ['Tenure Months', 'Monthly Charges', 'Total Charges']
fig, ax = plt.subplots(figsize=(15, 9))
ax = [plt.subplot(221), plt.subplot(222), plt.subplot(223)]
plt.subplots_adjust(hspace=0.3)
for i in range(0, len(ax)):
    ax[i].set_title("Distribution of " + num_features[i] + " by Churn")
    ax[i].set_ylabel('Density')
    ax[i].set_xlabel(num_features[i])
    sns.kdeplot(df[df['Churn Value'] == 1][num_features[i]], color= 'red', label= 'Churn Value: 1', ax=ax[i])
    sns.kdeplot(df[df['Churn Value'] == 0][num_features[i]], color= 'blue', label= 'Churn Value: 0', ax=ax[i])
plt.show()

OBSERVATIONS:                                                                                     
                  -  La plus part des Churners ont quitter des les premiers mois de Contract.                                
                   - Il y a des gens qui restent dans cette entreprise depuis environ 70 mois. Mais les clients qui ont enregistré des services auprès de l'entreprise entre 1 et 8 ans sont susceptibles de quitter l'entreprise et de mettre fin aux services.
-Le taux de churn augmente lorsque Monthly charges augmente.
                            
 -La distribution de Total Chages est le resultat des deux autres distributions(Total Charges = Monthly Charges * Tenure Months) 


### b- Varaibles qalitatives:

In [ ]:

#fonction qui calcul le taux de churn pour chaque categories
def plotFeature(feature, xoffset, yoffset):
    df1 = df.groupby(feature)["Churn Value"].value_counts().to_frame()
    df1 = df1.rename({"Churn Value": "Count"}, axis=1).reset_index()
    df1["Percentage"] = df1["Count"]/len(df)*100
    
    ax = sns.barplot(x=feature, y="Percentage", hue="Churn Value", data=df1)
    ax.set_title("Churn percentages for each " + feature)
    for p in ax.patches:
        height = p.get_height()
        ax.text(p.get_x() + p.get_width()/2 - xoffset,
                height - yoffset,
                '{:1.1f}%'.format(height),
                weight='bold', color='white', size = 12) 

#### GENDER - SENIORCITIZEN

In [ ]:
fig, ax = plt.subplots(figsize=(13,5))
plt.subplot(121)
plotFeature("Gender", 0.12, 4.5)
plt.subplot(122)
plotFeature("Senior Citizen", 0.11, 4.5)

plt.show()

OBSERVATIONS:                
-La distribution de Gender est homogéne donc elle n'a pas de relation avec le Churn

-on remarque aussi que lorsque Senir Citzen=No :leur taux de churn est superieur a cuex qui ont Senior Citizen=Yes

#### PARTNER-DEPENDENTS

In [ ]:
fig, ax = plt.subplots(figsize=(13,5))
plt.subplot(121)
plotFeature("Partner", 0.12, 4)
plt.subplot(122)
plotFeature("Dependents", 0.12, 3.5)
plt.show()

OBSERVATIONS:    
     -on remaque ceux qui n'ont pas de partenaires leur taux de Churn est superieur a ceux qui ont un partenaire                  
                           -les clients qui n'ont pas de dependaces ont un taux de churn superieur 

#### PHONE, INTERNET SERVICE, MULTIPLE LINE

In [ ]:
fig, ax = plt.subplots(figsize=(18,5))
plt.subplot(131)
plotFeature("Phone Service", 0.12, 3)
plt.subplot(132)
plotFeature("Internet Service", 0.18, 2)
plt.subplot(133)
plotFeature("Multiple Lines", 0.18, 2)
plt.show()

OBSERVATIONS     
         -les clients qui n'ont pas d'internet on a un taux de Churn inferieur a ceux qui ont acces a internet et les clients qui possedent des fibres
optiques ont le taux de churn le plus élevé

-Multiple Lines n'a pas vraiment d'influence

-les clients qui possedent un Phone sevcice leur taux de Churn est superieur aux autres

#### ONLINE SECURITY, ONLINE BACKUP, DEVICE PROTECTION

In [ ]:
fig, ax = plt.subplots(figsize=(18,5))
plt.subplot(131)
plotFeature("Online Security", 0.18, 2)
plt.subplot(132)
plotFeature("Online Backup", 0.18, 2)
plt.subplot(133)
plotFeature("Device Protection", 0.18, 2)
plt.show()

OBSERVATIONS:      
clients qui ne disposent pas de ces 3 services (Online Security, Online Backup and Device Protection) a quitter

#### TECH SUPPORT, STREAMING TV, STREAMING MOVIES

In [ ]:
fig, ax = plt.subplots(figsize=(18,5))
plt.subplot(131)
plotFeature("Tech Support", 0.18, 1.5)
plt.subplot(132)
plotFeature("Streaming TV", 0.18, 1.5)
plt.subplot(133)
plotFeature("Streaming Movies", 0.18, 1.5)
plt.show()

OBSERVATIONS:                                                                     
-Les clients qui ne disposent pas d'un support technique sont plus susceptibles a quitter
      
-Le service de streaming Tv ou MOVIES n'est pas prédictif du désabonnement 

#### CONTRACT - PAPERLESS BILLING - PAYMENT METHOD

In [ ]:
fig, ax = plt.subplots(figsize=(16,10))
plt.subplots_adjust(hspace=0.2)
plt.subplot(221)
plotFeature("Contract", 0.16, 1.5)
plt.subplot(222)
plotFeature("Paperless Billing", 0.1, 2)
plt.subplot(223)
plotFeature("Payment Method", 0.18, 1.5)
plt.show()

OBSERAVTAIONS:

-Les contrats intuitivement à court terme ont un taux de désabonnement beaucoup plus élevé (Contract)

-Les clients avec facturation sans papier ont un taux de désabonnement élevé(Paperless Billing)

-La méthode de paiement du chèque électronique montre un taux de désabonnement beaucoup plus élevé que les autres(Payment Method)

In [ ]:
df.shape

# Data Pre-processing

## 1-les valeurs extremes:

les variables ne suivent pas une distribution normale donc on peut utiliser la  Methode IQR pour trouver les valerus aberantes (outliers)

In [ ]:
for i in num_features:
    #calculer les quartiles
    Q1 = df[i].quantile(0.25)
    Q3 = df[i].quantile(0.75)
    IQR = Q3 - Q1
    #calculer les limites des valeurs aberantes
    lower_limit = Q1 - 1.5*IQR
    upper_limit = Q3 + 1.5*IQR
    #identifier ces valeurs
    print(((df[i] < lower_limit) | (df[i] > upper_limit)).any())

En utlisant la methode IQR on n'a pas de valeurs aberantes numerique detectés 

## 2-Encodage des donnés qualitatives:

In [ ]:
#creation d'un indicateur pour la variable gender en ululisant juste une colonne
df = pd.get_dummies(df, columns=['Gender'], drop_first=True)

# pour ces variables en va transformer les "yes" et "no "en 0 et 1
vars1 = ['Partner', 'Dependents', 'Phone Service', 'Paperless Billing','Senior Citizen']
map1 = {'No': 0, 'Yes': 1}
for i in vars1:
    df[i] = df[i].map(map1).astype('int')
    
#pour les variables ci_dessus on va utliser l'encodage one-hot puisque on a plus de deux categories dans chaque variables 
vars2 = ['Multiple Lines', 'Internet Service', 'Online Security', 'Online Backup', 'Device Protection', 'Tech Support',
         'Streaming TV', 'Streaming Movies', 'Contract', 'Payment Method']
df = pd.get_dummies(df, columns=vars2)
df.shape


In [ ]:
df

## 3-MinMax Scaler

In [ ]:
#les variavbles sont: tenure monthes, Monthly Charges and Total Charges
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[num_features] = scaler.fit_transform(df[num_features])

## 4-correlation entre les variables

In [ ]:
def corr_for_target(data, target, title=None):
    plt.figure(figsize=(4,14))
    sns.set(font_scale=1)
    
    sns.heatmap(data.corr()[[target]].sort_values(target, ascending=False)[1:], annot=True, cmap="coolwarm")
    
    if title: plt.title(f"\n{title}\n", fontsize=18)
    return

In [ ]:
corr_for_target(df, 'Churn Value');


on remarque ci-dessus  monthly contracts et l'absence de certains services
(online security, techsupport) sont corrolés positivement avec churn Value par contre
 tenure months ,  contract_two year et l'absence du service internet sont corrolés negativement avec churn Value

# Modelisation

In [6]:
from sklearn.model_selection import train_test_split, cross_val_score, KFold, cross_validate
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.metrics import plot_confusion_matrix, accuracy_score, precision_recall_fscore_support
from sklearn.metrics import plot_roc_curve, plot_precision_recall_curve

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [ ]:
X = df.drop(["Churn Value"], axis=1)
y = df["Churn Value"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=10)

In [ ]:
#insialisaton d'un dataframe qui garde la trace des modeles
all_scores = pd.DataFrame()

## 1-les fonctions utiles

In [ ]:
#Cette fonction prend le modèle, les caractéristiques et les valeurs cibles réelles, fait des prédictions,
#puis imprime différents scores métriques
def model_predictions(name, model, X, y, title):
    y_pred = model.predict(X)
    accuracy = accuracy_score(y, y_pred)
    precision, recall, f1, support = precision_recall_fscore_support(y, y_pred, average='binary')
    scores = pd.DataFrame(columns=['Accuracy','Precision','Recall','F1'])
    scores.loc[name] = [accuracy, precision, recall, f1]
    print('----------------------------------------')
    print(title)
    print(scores)
    return scores

In [ ]:
# Cette fonction trace la matrice de confusion avec / sans normalisation + courbe ROC 
def model_plots(model, X, y, title):
    fig, axes = plt.subplots(2, 2, figsize=(12, 8))
    fig.suptitle(title, fontsize=18)
    plt.subplots_adjust(hspace=0.3)
    axes[0,0].set_title('Confusion Matrix')
    plot_confusion_matrix(model, X, y, display_labels=['No Churn','Churn'], ax=axes[0,0])
    axes[0,1].set_title('Normalized Confusion Matrix')
    plot_confusion_matrix(model, X, y, normalize='true', display_labels=['No Churn','Churn'], ax=axes[0,1])
    axes[1,0].set_title('ROC Curve')
    plot_roc_curve(model, X, y, name='Log.Reg.', ax=axes[1,0])
    #axes[1,1].set_title('Precision-Recall Curve')
    #plot_precision_recall_curve(model, X, y, name='Log.Reg.', ax=axes[1,1])
    plt.show()

In [ ]:
# Cette fonction prend le modèle et ses paramètres associés en entrée et renvoie les meilleurs paramètres
# à partir d'un ensemble donné à l'aide de l'algorithme de GridSearchCV (réglage des hyperparamètres)
def hyperparam_gridcv(name, model, params):
    gsearch = GridSearchCV(estimator=model, 
                           param_grid=params, 
                           scoring='f1',
                           cv=5,
                           n_jobs = -1)
    gsearch.fit(X_train, y_train)
    print(name, " - Best Parameters: ", gsearch.best_params_)
    return gsearch.best_params_

In [7]:
# Cette fonction prend le modèle et ses paramètres associés en entrée et renvoie les meilleurs paramètres
# à partir d'un ensemble donné à l'aide de l'algorithme de recherche aléatoire
def hyperparam_randcv(name, model, params):
    gsearch = RandomizedSearchCV(estimator=model, 
                                 param_distributions=params, 
                                 n_iter = 100,
                                 scoring='f1',
                                 cv=5,
                                 n_jobs = -1)
    gsearch.fit(X_train, y_train)
    print(name, " - Best Parameters: ", gsearch.best_params_)
    return gsearch.best_params_

## a-la regression logistique

In [ ]:
# Ajuster le modèle de base
logreg = LogisticRegression(max_iter=1000, random_state=10)
logreg.fit(X_train, y_train)

In [ ]:
# Obtenez les résultats du train et du test
model_predictions('LOGREG', logreg, X_train, y_train, 'Logistic Reg. Training Set Scores')
model_predictions('LOGREG', logreg, X_test, y_test, 'Logistic Reg. Test Set Scores')
# Tracer la matrice de confusion, courbe ROC
model_plots(logreg, X_test, y_test, "Logistic Regression Results (Base Model)")

In [ ]:
# Réglage des hyperparamètres (il est commenté car cela prend du temps)
params_logreg = {'penalty':['l1','l2'], 'C':np.arange(0.1, 3, 0.1)}
#hyperparam_gridcv('Logistic Regression', logreg, params_logreg)

In [ ]:
logreg = LogisticRegression(max_iter=1000, C=1.1, penalty='l2', random_state=10)
logreg.fit(X_train, y_train)

logreg_results = model_predictions('LOGREG', logreg, X_test, y_test, 'Logistic Reg. Test Set Scores')
all_scores = all_scores.append(logreg_results)

model_plots(logreg, X_test, y_test, "Logistic Regression Results (Tuned Model)")

## b-KNN

In [ ]:
# Ajuster le modèle de base
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)

In [ ]:
# Obtenez les résultats du train et du test

model_predictions('KNN', knn, X_train, y_train, 'KNN Training Set Scores')
model_predictions('KNN', knn, X_test, y_test, 'KNN Test Set Scores')
# Tracer la matrice de confusion, courbe ROC
model_plots(knn, X_test, y_test, "KNN Results (Base Model)")

In [ ]:
# Réglage des hyperparamètres
params_knn = {'n_neighbors':np.arange(3, 25, 1), 
              'metric':['euclidean','manhattan','minkowski']}
#hyperparam_gridcv('K-Nearest Neighbors', knn, params_knn)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=19, metric='manhattan')
knn.fit(X_train, y_train)

knn_results = model_predictions('KNN', knn, X_test, y_test, 'KNN Test Set Scores')
all_scores = all_scores.append(knn_results)

model_plots(knn, X_test, y_test, "KNN Results (Tuned Model)")

## c-Random forest

In [ ]:
# Ajuster le modèle de base
rf = RandomForestClassifier(random_state=10)
rf.fit(X_train, y_train)

In [ ]:
# Obtenez les résultats du train et du test

model_predictions('RND.FOR.', rf, X_train, y_train, 'Random Forest Training Set Scores')
model_predictions('RND.FOR.', rf, X_test, y_test, 'Random Forest Test Set Scores')
# Tracer la matrice de confusion, courbe ROC
model_plots(rf, X_test, y_test, "Random Forest Results (Base Model)")
 #0.794653   0.591379  0.734475  0.655205

In [ ]:
rf?

In [ ]:
# Réglage des hyperparamètres
params_rf = {'n_estimators':[100, 500, 1000, 1500],
             'max_depth':np.append(np.arange(10, 60, 10), None),
             'class_weight':[None, 'balanced', 'balanced_subsample'],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 5]
            }

#hyperparam_gridcv('Random Forest', rf, params_rf)

In [ ]:
rf = RandomForestClassifier(n_estimators=100, 
                            max_depth=10, 
                            max_features='auto', 
                            min_samples_split=5,
                            min_samples_leaf=2, 
                            class_weight='balanced_subsample',
                            random_state=10)
rf.fit(X_train, y_train)

rf_results = model_predictions('RND.FOR.', rf, X_test, y_test, 'Random Forest Test Set Scores')
all_scores = all_scores.append(rf_results)

model_plots(rf, X_test, y_test, "Random Forest Results (Tuned Model)")

In [ ]:
#l'importance des paramétres
features = pd.DataFrame()
features['feature'] = X.columns
features['importance'] = rf.feature_importances_
features.sort_values(by=['importance'], ascending=True, inplace=True)
features.set_index('feature', inplace=True)

features.plot(kind='barh', figsize=(10, 10))

OBSERVATIONS

le modele ajusté a une perfomance legemerment superieur aux deux premiers algorithmes
les premiers alogorithmes peuvent predire que 50% des clients qui vont quitter parcontre le random Forest peut predire jusqu'a 75% des churners.

D'apres features Importance Tenure monthes,total charges, contact month_to_month and monthly charges sont les variables les plus importantes dans la prediction

## d-SUPPORT VECTOR MACHINES

In [ ]:
# Ajuster le modèle de base
svm = SVC(random_state=10)
svm.fit(X_train, y_train)

In [ ]:
# Obtenez les résultats du train et du test
model_predictions('SVM', svm, X_train, y_train, 'SVM Training Set Scores')
model_predictions('SVM', svm, X_test, y_test, 'SVM Test Set Scores')
# Tracer la matrice de confusion, courbe ROC
model_plots(svm, X_test, y_test, "SVM Results (Base Model)")

In [ ]:
# Réglage des hyperparamètres
params_svm = {'C':np.arange(0.1, 3, 0.1), 'kernel':['linear', 'poly', 'rbf']}
#hyperparam_gridcv('SVM', svm, params_svm)

In [ ]:
svm = SVC(C=0.3,
          kernel='linear',
          random_state=10)
svm.fit(X_train, y_train)

svm_results = model_predictions('SVM', svm, X_test, y_test, 'SVM Test Set Scores')
all_scores = all_scores.append(svm_results)

model_plots(svm, X_test, y_test, "SVM Results (Tuned Model)")

OBSERVATION

Le modèle ajusté a des résultats légèrement meilleurs que le modèle de base, mais dans l'ensemble, 
il est médiocre pour prédire le taux de désabonnement des clients comme la régression logistique et le modèle KNN.

## f-XGBOOST

In [9]:
# Ajuster le modèle de base
xgb = XGBClassifier(random_state=10)
#xgb.fit(X_train, y_train)

In [ ]:
# Obtenez les résultats du train et du test
model_predictions('XGB', xgb, X_train, y_train, 'XGB Training Set Scores')
model_predictions('XGB', xgb, X_test, y_test, 'XGB Test Set Scores')
# Tracer la matrice de confusion, courbe ROC
model_plots(xgb, X_test, y_test, "XGB Results (Base Model)")

In [10]:
# Réglage des hyperparamètres
params_xgb = {'learning_rate': [0.01, 0.05, 0.1],
              'max_depth': [3, 5, 7, 10], 
              'min_child_weight': [1, 5, 10],
              'subsample': [0.5, 0.7, 0.9],
              'colsample_bytree': [0.6, 0.8, 1.0],
              'gamma': [0, 0.05, 0.1],
              'scale_pos_weight':[1,2,3]}
#hyperparam_randcv('XGB', xgb, params_xgb)

NameError: name 'X_train' is not defined

In [ ]:
xgb = XGBClassifier(n_estimators=100,
                    learning_rate=0.05,
                    max_depth=3,
                    min_child_weight=10,
                    subsample=0.5,
                    colsample_bytree=0.8,
                    gamma=0.05,
                    scale_pos_weight=2,
                    random_state=10)
xgb.fit(X_train, y_train)

xgb_results = model_predictions('XGB', xgb, X_test, y_test, 'XGB Test Set Scores')
all_scores = all_scores.append(xgb_results)

model_plots(xgb, X_test, y_test, "XGB Results (Tuned Model)")

## e-LIGHT GBM

In [ ]:
# Ajuster le modèle de base
lgbm = LGBMClassifier()
lgbm.fit(X_train, y_train)

In [ ]:
# Obtenez les résultats du train et du test
model_predictions('LGBM', lgbm, X_train, y_train, 'LGBM Training Set Scores')
model_predictions('LGBM', lgbm, X_test, y_test, 'LGBM Test Set Scores')
# Tracer la matrice de confusion, courbe ROC
model_plots(lgbm, X_test, y_test, "LGBM Results (Base Model)")

In [ ]:
# Réglage des hyperparamètres
params_lgbm = {'learning_rate': [0.01, 0.05, 0.1],
               'n_estimators' : [100, 1000, 2000],
               'max_depth': [-1, 3, 5, 7],
               'min_child_weight': [1, 10, 100],
               'subsample': [0.5, 0.7, 0.9],
               'colsample_bytree': [0.6, 0.8, 1.0],
               'reg_alpha': [0, 1, 5, 10],
               'scale_pos_weight':[1, 1.5, 2]}
#hyperparam_randcv('LGBM', lgbm, params_lgbm)

In [ ]:
lgbm = LGBMClassifier(n_estimators=2000,
                    learning_rate=0.1,
                    min_child_weight=100,
                    max_depth=3,
                    subsample=0.5,
                    colsample_bytree=1,
                    scale_pos_weight=2,
                    reg_alpha=10,
                    random_state=10)
lgbm.fit(X_train, y_train)

lgbm_results = model_predictions('LGBM', lgbm, X_test, y_test, 'LGBM Test Set Scores')
all_scores = all_scores.append(lgbm_results)

model_plots(lgbm, X_test, y_test, "LGBM Results (Tuned Model)")

In [ ]:
#all_scores.sort_values(by='F1', ascending=False)

### 7-Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

In [ ]:
MultinomialNB?

In [ ]:
# Entrainer le modele pour les trois type d'algorithme de Naive Bayes:
nb={'gaussian':GaussianNB(),'Bernoulli':BernoulliNB(),'Multinomial': MultinomialNB()}
score={}
for key,model in nb.items():
    s=cross_val_score(model,X_train,y_train,cv=5, scoring='accuracy')
    score[key]=np.mean(s)
score

In [ ]:
NaiveMulti= MultinomialNB()
NaiveMulti.fit(X_train,y_train)

In [ ]:
#Les prédictions
y_pred=NaiveMulti.predict(X_test)
#évaluation de notre modèle
print ("précision: ", accuracy_score(y_test, y_pred))
print ("test score: ", NaiveMulti.score(X_test, y_test))

In [ ]:
NaiveMulti_results=model_predictions('NaiveMulti', NaiveMulti, X_test, y_test, 'NaiveMulti. Test Set Scores')
model_plots(NaiveMulti, X_test, y_test, "NaiveMulti  Model")

In [ ]:
all_scores = all_scores.append(NaiveMulti_results)

In [ ]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

sbfs = SFS(model, 
          k_features=1, 
           forward=False, 
           floating=True, 
           verbose=2,
           scoring='accuracy',
           cv=5,
           n_jobs=-1)

sbfs = sbfs.fit(X_train, y_train)

In [ ]:
pd.DataFrame.from_dict(sbfs.get_metric_dict()).T

In [ ]:
print('\nSFS 1 :')
print(sbfs.k_feature_idx_)
print('CV Score:')
print(sbfs.k_score_)
print('CV Score:')
print(sbfs.k_feature_names_)

In [ ]:
sbfs = SFS(model, 
          k_features=2, 
           forward=False, 
           floating=True, 
           verbose=2,
           scoring='accuracy',
           cv=5,
           n_jobs=-1)

sbfs = sbfs.fit(X_train, y_train)

In [ ]:
pd.DataFrame.from_dict(sbfs.get_metric_dict()).T

In [ ]:
print('\nSFS 2 :')
print(sbfs.k_feature_idx_)
print('CV Score:')
print(sbfs.k_score_)
print('CV Score:')
print(sbfs.k_feature_names_)

In [ ]:
sbfs = SFS(model, 
          k_features=3, 
           forward=False, 
           floating=True, 
           verbose=2,
           scoring='accuracy',
           cv=5,
           n_jobs=-1)

sbfs = sbfs.fit(X_train, y_train)

In [ ]:
pd.DataFrame.from_dict(sbfs.get_metric_dict()).T

In [ ]:
print('\nSFS 3 :')
print(sbfs.k_feature_idx_)
print('CV Score:')
print(sbfs.k_score_)
print('CV Score:')
print(sbfs.k_feature_names_)

In [ ]:
sbfs = SFS(model, 
          k_features=4, 
           forward=False, 
           floating=True, 
           verbose=2,
           scoring='accuracy',
           cv=5,
           n_jobs=-1)

sbfs = sbfs.fit(X_train, y_train)

In [ ]:
print('\nSFS 4 :')
print(sbfs.k_feature_idx_)
print('CV Score:')
print(sbfs.k_score_)
print('CV Score:')
print(sbfs.k_feature_names_)

In [ ]:
model.fit(X_train,y_train)# modèle d' apprentissage automatique se généralise à des données similaires à celles sur lesquelles i

In [ ]:
plot_confusion_matrix(model, X_test, y_test,cmap=plt.cm.Blues)  
plt.show()

In [ ]:
X_train_nb2=X_train[['Dependents','Tenure Months', 'Internet Service_Fiber optic', 'Contract_Month-to-month']].copy()
X_test_nb2=X_test[['Dependents','Tenure Months', 'Internet Service_Fiber optic', 'Contract_Month-to-month']].copy()

model2=GaussianNB()
model2 = model2.fit(X_train_nb2, y_train)

print('train score ')
print('Befor', model.score(X_test,y_test),'  After:', model2.score(X_train_nb2,y_train))
print('test score ')
print('Befor :', model.score(X_test,y_test),'  After: ',model2.score(X_test_nb2,y_test))

# 6-évaluation

In [ ]:
all_scores.sort_values(by='F1', ascending=False)

le meuilleur algorithme est light GBM car il a le meuilleur score F1